*Getting Started with Jupyter and FPGA-SPICE on Linux
1. Install Anaconda: https://conda.io/docs/user-guide/install/linux.html 
2. Run anaconda-navigator: `~/anaconda-navigator`
3. Launch jupyter notebook from anaconda navigator

In the jupyter interface, navigate to the location of a notebook and view it.

The repository contains a submodule, so you need to get it as well:

Build the VPR tool with the FPGA-SPICE functionality:

In [38]:
import os
# move to the directory with the Makefile
os.chdir("./tangxifan-eda-tools/branches/vpr7_rram/vpr/")
# run make
print ("Wait for make to finish with a return code of 0.")
os.system("make")

Wait for make to finish with a return code of 0.


0

To run FPGA-SPICE, define its location:

In [39]:
# set this to the location of the vpr executable file
fpga_spice = "~/OpenFPGA/tangxifan-eda-tools/branches/vpr7_rram/vpr/vpr"

Define an architecture file and a circuit file:

In [40]:
arch = "./tutorial/example_arch.xml"
circuit = "./tutorial/example_circuit.blif"

Run vpr:

In [41]:

command_line = fpga_spice + " " + arch + " " + circuit
os.system(command_line)

256

This will bring up a graphical display of how the circuit is being placed on the FPGA. Press the `Proceed` button to step to the final placement, press `Proceed` again to step to the routing. Press the `Exit` button to exit the display.

To run VPR without the display, use the command `-nodisp`

In [23]:
command_line_nodisp = command_line + " -nodisp"


In [36]:
os.system(command_line_nodisp)

0

When you ran the tool, a number of text output files are created that report on what the tool did. Open the file "OpenFPGA/tutorial/example_circuit.place" in a text editor and note on line 2: "Array size: 2 x 2 logic blocks". This is a report that when the tool placed and routed the logic, it ended up in a 2 x 2 grid.

Now open the file "OpenFPGA/tutorial/example_arch.xml" in a text editor. Go to line 51 and change:

<pb_type name="io" capacity="1">

to:

<pb_type name="io" capacity="3">

Now, instead of each input/output block only having 1 input or output, they can have 3 inputs or outputs.

Run the tool again:

In [34]:
os.system(command_line_nodisp)

0

Re-open the file "OpenFPGA/tutorial/example_circuit.place" in a text editor and note that the logic is now placed in one block.

The simple change to the number of inputs or outputs per I/O block let the placement and routing tool use less space on the FPGA for the circuit.